# Test Sum-base
Draft version.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
import nltk
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import logging
import os
from tensorboardX import SummaryWriter

/Users/junwenbu/opt/anaconda3/envs/sum/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/junwenbu/opt/anaconda3/envs/sum/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/junwenbu/opt/anaconda3/envs/sum/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/junwenbu/opt/anaconda3/envs

In [2]:
wikihow = pd.read_csv("data/clean_wikihow.csv")

In [3]:
wikihow.head()

,summary,text
0,sell yourself first,anything else stop sum artist think translate ...
1,read the classics before 1600,reading classics first thing well read want bu...
2,join online artist communities,depending scale intend sell art pieces may wan...
3,make yourself public,get best advertising publish example pieces ar...
4,blog about your artwork,given hundreds free blogging websites lot choi...


## Make sure the settings are the same as your training script

In [4]:
summaries = wikihow['summary'].tolist()
texts = wikihow['text'].tolist()
print(len(summaries), wikihow.shape)

# dataset range
si, ei = 0, 10
# Subset the data for training
start = 0 #0
end = start + 10 #20000

summaries = summaries[si:ei]
texts = texts[si:ei]

# Set the Hyperparameters
epochs = 100
batch_size= 4 #64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

1212030 (1212030, 2)


In [5]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in str(sentence).split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [6]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, summaries)
count_words(word_counts, texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 573


In [7]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('numberbatch-en-19.08.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 516783


In [8]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 10

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 0
Percent of words that are missing from vocabulary: 0.0%


In [9]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 573
Number of words we will use: 554
Percent of words we will use: 96.67999999999999%


## Test

In [10]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [12]:
tests = [0, 1, 2, 3]

checkpoint = "checkpoints/best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    for t in tests:
        text = text_to_seq(texts[t])
        #Multiply by batch_size to match the model's input parameters
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

        # Remove the padding from the tweet
        pad = vocab_to_int["<PAD>"]
        # print('Original Text:', wikihow.text[random])
        # print('Original summary:', wikihow.summary[random])#clean_summaries[random]
        print('\n==================== Example {} ===================='.format(t))
        print('Original Text:')
        print(texts[t])
        print('\nOriginal Summary:')
        print(summaries[t])
        print('\nInput:')
        print('Word Ids: {}'.format([i for i in text]))
        print('{}'.format(" ".join([int_to_vocab[i] for i in text])))
        print('\nOutput Summary:')
        print('Word Ids: {}'.format([i for i in answer_logits if i != pad]))
        print('{}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from checkpoints/best_model.ckpt

==================== Example 0 ====================
Original Text:
anything else stop sum artist think translate online profile words twitter allows entire page indulgence website would allow bring salient features creativity experience passion reasons painting make clear readers artist loves art produces high quality art true champion art great words find friend help really important aspect selling online – establishment credibility reliability

Original Summary:
sell yourself first 

Input:
Word Ids: [46, 47, 48, 49, 9, 50, 51, 8, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 11, 70, 71, 9, 72, 29, 73, 74, 75, 29, 76, 77, 29, 78, 53, 79, 80, 81, 82, 83, 84, 85, 8, 550, 86, 87, 88]
anything else stop sum artist think translate online profile words twitter allows entire page indulgence website would allow bring salient features creativity experience passion reasons painting make clear reade